In [1]:
import os
import sys
import xgboost
import shap
import argparse
import pickle as pkl

import matplotlib.pyplot as plt

from scipy import stats
from utilities import *
from models import *
from sklearn import preprocessing
from shapreg import removal, games, shapley

#### import CATE model
module_path = os.path.abspath(os.path.join('CATENets/'))

if module_path not in sys.path:
    sys.path.append(module_path)

from catenets.models.jax import TNet, SNet,SNet1, SNet2, SNet3, DRNet, RANet, PWNet, RNet, XNet
import catenets.models as cate_models    

/homes/gws/mingyulu/miniconda3/envs/torch_gpu/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
n = 1000
feature_size = 5
random_state = 0

X, y_po, w, p, KL = simulation(feature_size, n, 0, 0, random_state, False)

min_max_scaler = preprocessing.MinMaxScaler()
X_scaled = min_max_scaler.fit_transform(X)

#print(KL)
rng = np.random.default_rng(random_state)
inds = np.arange(n)
rng.shuffle(inds)

n_train = int(0.8 * n)

train_inds = inds[:n_train]
test_inds = inds[n_train:]

x_oracle_train = torch.from_numpy(X_scaled[train_inds,:])
w_oracle_train = torch.from_numpy(w[train_inds,:])
y_oracle_train = torch.from_numpy(np.take_along_axis(y_po,w, 1)[train_inds, :])
y_test_cate = y_po[test_inds, 1] - y_po[test_inds, 0]

### Create Cate model 

torch_DR = cate_models.torch.DRLearner(
            2*feature_size,
            binary_y=(len(np.unique(y_po)) == 2),
            n_layers_out=2,
            n_units_out=100,
            n_iter=1000,
            batch_size=128,
            batch_norm=False,
            nonlin="relu",
            )

cate_model = Cate(torch_DR, MaskLayer())

### Train model with maskes

cate_model.fit(x_oracle_train ,y_oracle_train, w_oracle_train)

x_oracle_test = torch.from_numpy(X_scaled[test_inds,:])
w_oracle_test = torch.from_numpy(w[test_inds,:])
test_mask = torch.ones(x_oracle_test.size()[0],x_oracle_test.size()[1])
test_phe = cate_model.predict(x_oracle_test, test_mask).cpu().detach().numpy()

print("phe is %s" %mse(test_phe, y_test_cate))

#### Explanation method

# Make model callable
model_lam = lambda x, x_mask: cate_model.predict(x, x_mask)

# Model extension

instance = torch.reshape(torch.from_numpy(X_scaled[0, :]), (1,-1))

game  = games.CateGame(instance, cate_model)
explanation = shapley.ShapleyRegression(game, batch_size=32)

# Plot with 95% confidence intervals
names = ["feature " + str(i) for i in range(feature_size)]

covariance is not positive-semidefinite.
covariance is not positive-semidefinite.


phe is 7.255140121916818


 51%|█████▏    | 0.513/1 [09:18<08:49, 1088.24s/it] 